# Chatbot with LLM and RAG

A book and movie assistant chatbot using Python, LangChain, embedding-based retrieval strategies, and OpenAI's GPT-3.5

# Step 1: Preparation of the Corpus
- The book corpus is based on the [CMU Book Summary Corpus](https://www.kaggle.com/datasets/ymaricar/cmu-book-summary-dataset).
- The movie corpus is based on the [CMU Movie Summary Corpus](https://www.cs.cmu.edu/~ark/personas/).
- For simplicity, I've already processed and saved the corpus snippets into **data/BookSummaries/book.json** and **data/MovieSummaries/movie.json**. If you prefer, you can download the original corpus and parse it yourself.

# Step 2 Vector Space
- Embedding-Based Retrieval: LangChain supports various retrieval strategies. If LangChain's built-in retrieval mechanisms do not mean your embedding-based retrieval requirement, you can integrate LlamaIndex or a similar tool that can create and query embeddings of your corpus.

In [1]:
# %load vector_space.py
# vector_space.py
import json
from pathlib import Path
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from tqdm import tqdm

class EmbeddingManager:
    def __init__(self, model_name):
        self.embeddings = HuggingFaceEmbeddings(model_name=model_name)
    
    def get_embeddings(self):
        return self.embeddings

class VectorSpaceManager:
    def __init__(self, embedding_manager):
        self.embedding_manager = embedding_manager
        self.embeddings = self.embedding_manager.get_embeddings()

    def create_vector_space(self, documents):
        vector_store = FAISS.from_documents(documents[:2], self.embeddings)

        with tqdm(total=len(documents), desc="Creating vector space") as pbar:
            batch_size = 100
            for i in range(2, len(documents), batch_size):
                batch_documents = documents[i:i+batch_size]
                tempt_vector_store = FAISS.from_documents(batch_documents, self.embeddings)
                vector_store.merge_from(tempt_vector_store)
                pbar.update(len(batch_documents))

        return vector_store

    def save_vector_space(self, vector_store, save_path):
        print(f"Saving vector space to {save_path}...")
        vector_store.save_local(save_path)
        print(f"Finished!")

    def load_vector_space(self, save_path):
        print(f"Lodaing vector space from {save_path}")
        return FAISS.load_local(save_path, self.embeddings)


class DataLoader:
    def __init__(self, json_file_path):
        self.json_file_path = json_file_path

    def load_data(self):
        data = json.loads(Path(self.json_file_path).read_text())
        return data

    def create_documents(self, length=None):
        data = self.load_data()
        if length is None:
            length = len(data)
        
        documents = [
            Document(
                page_content=self.get_page_content(item),
                metadata=item
            )
            for item in data[:length]
        ]
        return documents

    def get_page_content(self, item):
        raise NotImplementedError("Subclasses must implement get_page_content method")
    

class BookDataLoader(DataLoader):
    def get_page_content(self, item):
        return f"{item['title']} {item['author']} {item['publication_date']} {item['description']} {' '.join(item['genres'])}"
    

class MovieDataLoader(DataLoader):
    def get_page_content(self, item):
        # {movie_id, title, release_date, supported_languages, movie_countries, movie_genres_list, movie_actor_list, summary}
        # Use all the fields to create the page content
        return f"{item['title']} {item['release_date']} {item['summary']} {' '.join(item['movie_genres_list'])} {' '.join(item['movie_actor_list'])}"

def process_data(json_file_path, model_name, save_path, data_loader_class, length=None):
    # Initialize the embedding manager with the chosen model
    embedding_manager = EmbeddingManager(model_name)

    # Initialize the vector space manager with the embedding manager
    vector_space_manager = VectorSpaceManager(embedding_manager)

    # Load data and create documents
    data_loader = data_loader_class(json_file_path)
    documents = data_loader.create_documents(length=length)

    # Create and save the vector space
    vector_store = vector_space_manager.create_vector_space(documents)
    vector_space_manager.save_vector_space(vector_store, save_path)

    # Load the vector space and perform a search
    vector_store = vector_space_manager.load_vector_space(save_path)
    query = "The Hobbit"
    search_results = vector_store.search(query, k=2, search_type="similarity")
    print(search_results)

# Example usage
if __name__ == "__main__":
    # Book
    json_file_path = 'data/BookSummaries/book.json'  # Replace with the actual JSON file path
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    save_path = 'data/book_vector_store'  # Replace with the actual save path
    process_data(json_file_path, model_name, save_path, BookDataLoader, 100)

    # Movie
    json_file_path = 'data/MovieSummaries/movie.json'  # Replace with the actual JSON file path
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    save_path = 'data/movie_vector_store'  # Replace with the actual save path
    process_data(json_file_path, model_name, save_path, MovieDataLoader, 100)


# Step 3: Chatbot Logic and User Interaction
- User Query Handling: Design the chatbot to accept user input, which could range from specific questions about books to general requests for recommendations.
- Query to Embedding: Convert the user query into an embedding and use the retrieval strategy to find the most relevant book summaries.
- Interaction with GPT-3.5: Send the user query and retrieved book summaries to GPT-3.5 to generate a coherent and contextually appropriate response.
- Response Generation: Combine the LLM's output with the retrieved information to generate a final response to the user.

In [165]:
class TopicClassifier:
    def __init__(self, llm):
        """
        Initializes a TopicClassifier object.

        Parameters:
        llm (LanguageModel): The language model used for classification.

        Returns:
        None
        """
        self.llm = llm
        self.topics = ["movies", "books", "others"]


    def classify(self, query):
        """
        Classifies a given query into one of the predefined topics.

        Parameters:
        query (str): The query to be classified.

        Returns:
        str: The classified topic.
        """
        prompt = f"Classify the following question into one of these topics: '{','.join(self.topics)}': '{query}'"
        response = self.llm.predict(text=prompt, max_tokens=10)
        topic = response.strip().lower()
        return topic

In [166]:
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS  
from langchain.agents import Tool
from langchain import PromptTemplate

class ToolManager:
    def __init__(self, llm, movies_vector_path, books_vector_path, embeddings):
        self.llm = llm
        self.movies_vector_path = movies_vector_path
        self.books_vector_path = books_vector_path
        self.embeddings = embeddings
        self.tools = self._initialize_tools()

    def _initialize_tools(self):
        # Load FAISS vector stores for movies and books
        movies_vector_store = FAISS.load_local(self.movies_vector_path, self.embeddings)
        books_vector_store = FAISS.load_local(self.books_vector_path, self.embeddings)

        # Define prompt template
        prompt_template = """If the context is not relevant, 
        please answer the question by using your own knowledge about the topic
        
        {context}
        
        Question: {question}
        """
        PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

        # Initialize RetrievalQA tools with the FAISS vector stores
        movies_qa = RetrievalQA.from_chain_type(llm=self.llm, chain_type="stuff", retriever=movies_vector_store.as_retriever(search_kwargs={"k": 3}), chain_type_kwargs={"prompt": PROMPT})
        books_qa = RetrievalQA.from_chain_type(llm=self.llm, chain_type="stuff", retriever=books_vector_store.as_retriever(search_kwargs={"k": 3}), chain_type_kwargs={"prompt": PROMPT})

        # Return a dictionary of tools for movies and books
        return {
            "movies": Tool(name="MoviesTool", func=movies_qa.run, description="Retrieve movie information."),
            "books": Tool(name="BooksTool", func=books_qa.run, description="Retrieve book information.")
        }

    def get_tool(self, topic):
        return self.tools.get(topic)


In [167]:
from langchain.memory import ConversationBufferMemory
from langchain.agents import ConversationalChatAgent, AgentExecutor

class ChatAgent:
    def __init__(self, llm, tool_manager):
        self.llm = llm
        self.tool_manager = tool_manager
        self.memory = ConversationBufferMemory(memory_key="chat_history",input_key="input", return_messages=True)
        self.agent = ConversationalChatAgent.from_llm_and_tools(llm=self.llm, tools=list(self.tool_manager.tools.values()), system_message="You are a smart assistant whose main goal is to recommend amazing books and movies to users. Provide helpful, **short** and concise recommendations with a touch of fun!")
        self.chat_agent = AgentExecutor.from_agent_and_tools(agent=self.agent, tools=list(self.tool_manager.tools.values()), verbose=True, memory=self.memory)

    def get_response(self, query, topic_classifier):
        """
        Get the response from the chat agent based on the given query and topic classifier.

        Args:
            query (str): The user's query.
            topic_classifier (TopicClassifier): The topic classifier used to classify the query.

        Returns:
            dict: A dictionary containing the response generated by the chat agent.
                  The response is stored under the key "answer".
        """
        topic = topic_classifier.classify(query)
        tool_name = None if topic == "other" else topic.capitalize() + "Tool"

        try:
            response = self.chat_agent.run(input=query, tool_name=tool_name) if tool_name else self.llm.generate(prompt=query)
        except ValueError as e:
            response = str(e)

        return {"answer": response}

In [168]:
import os
from langchain.chat_models import ChatOpenAI

In [169]:
#  Retrieve the OpenAI API key and temperature from environment variables
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', 'YOUR_API_KEY')
OPENAI_TEMPERATURE = float(os.getenv('OPENAI_TEMPERATURE', '0.0'))

In [170]:
# Initialize the ChatOpenAI model
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model='gpt-3.5-turbo',
    temperature=OPENAI_TEMPERATURE,
)

In [171]:
from vector_space import EmbeddingManager

# Initialize components
book_vector_store_path = "data/book_vector_store"
movie_vector_store_path = "data/movie_vector_store"
embeddings_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = EmbeddingManager(embeddings_model_name).get_embeddings()
tool_manager = ToolManager(llm, movie_vector_store_path, book_vector_store_path,embeddings)
topic_classifier = TopicClassifier(llm)
chat_agent = ChatAgent(llm, tool_manager)

# Step 4: Testing and Iteration
- Prototype Testing: Test the chatbot with a range of queries to ensure it retrieves relevant information and that LLM generates appropriate responses.
- Iterative Improvement: Based on testing result, refine the retrieval strategy, prompt design, and response processing to improve the chatbot's accuracy and user experience.

In [173]:
print("Chatbot is ready to talk! Type 'quit' to exit.")
    
while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        break

    response = chat_agent.get_response(user_input, topic_classifier)
    print(f"You: {user_input}")
    print(f"Chatbot: {response['answer']}")

Chatbot is ready to talk! Type 'quit' to exit.


> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "I am a smart assistant here to recommend amazing books and movies to you!"
}
```

> Finished chain.
You: hello, who are you
Chatbot: I am a smart assistant here to recommend amazing books and movies to you!


> Entering new AgentExecutor chain...
```json
{
    "action": "BooksTool",
    "action_input": "Children's books for 2-year-olds"
}
```
Observation: Some popular children's books for 2-year-olds include "Goodnight Moon" by Margaret Wise Brown, "Brown Bear, Brown Bear, What Do You See?" by Bill Martin Jr. and Eric Carle, "The Very Hungry Caterpillar" by Eric Carle, "Where the Wild Things Are" by Maurice Sendak, and "Chicka Chicka Boom Boom" by Bill Martin Jr. and John Archambault. These books are engaging, colorful, and often have simple storylines that are perfect for young children.
Thought:```json
{
    "action": "Final Answer",
    